# How to transfer raw image data to TFRecords
----

Hello everyone! This tutorial, like the previous one, is focused on automatizing the data input pipeline.

Most of the time, our datasets are too big too read in memory so we have to prepare a pipeline for reading the data in batches from hard-disk. I always process my raw data (text, images, tabular) to TFRecords as it makes my life so much easier hehe :).

This tutorial will cover the following parts:
* create a function that reads raw images and transfers it to TFRecords.
* create a function that parses the TFRecords to TF tensors.

So without any further due, let's get started.

### Transfer raw images to TFRecords
----

For this task, we will be using a few images from the FER2013 dataset, that you can find in the datasets/dummy_images folder.

In [4]:
# Import TensorFlow and TensorFlow Eager
import tensorflow as tf
import tensorflow.contrib.eager as tfe

import glob

In [5]:
# Enable eager mode. Once activated it cannot be reversed! Run just once.
tfe.enable_eager_execution()

In [38]:
def img2tfrecords(path_data='datasets/dummy_images/', image_format='jpg'):
    '''
    '''
    def _int64_feature(value):
        return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
    
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    
    filenames = glob.glob(path_data + '*' + image_format)
    writer = tf.python_io.TFRecordWriter(path_data + 'dummy.tfrecords')
    
    
    for filename in filenames:
        with tf.gfile.FastGFile(filename, 'rb') as f:
            img = f.read()
        label = int(filename.split('_')[-1].split('.')[0])
        example = tf.train.Example(features=tf.train.Features(feature={
                                  'label': _int64_feature(label),
                                  'image': _bytes_feature(img)}))
    
        writer.write(example.SerializeToString())